先建立7层CNN的分类模型，使用 DepthWise 和 PointWise 替换传统卷积，获得降低计算复杂度的 Student 模型。
使用 gamma 系数对 Student 模型进行裁剪，得到简化结构后的模型

In [1]:
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import time

In [2]:
def readfile(path, label):
    image_dir = sorted(os.listdir(path))
    x = np.zeros((len(image_dir), 128, 128, 3), dtype=np.uint8)
    y = np.zeros((len(image_dir)), dtype=np.uint8)
    for i, file in enumerate(image_dir):
        print(os.path.join(path, file))
        img = cv2.imread(os.path.join(path, file))
        x[i, :, :] = cv2.resize(img,(128, 128))
        if label:
          y[i] = int(file.split("_")[0])
    if label:
      return x, y
    else:
      return x

In [3]:
workspace_dir = "G:/Model_Compress/Dataset"
print("Reading data")
train_x, train_y = readfile(os.path.join(workspace_dir, "training"), True)

Reading data
G:/Model_Compress/Dataset\training\0_200.png
G:/Model_Compress/Dataset\training\0_201.png
G:/Model_Compress/Dataset\training\0_202.png
G:/Model_Compress/Dataset\training\0_203.png
G:/Model_Compress/Dataset\training\0_204.png
G:/Model_Compress/Dataset\training\0_205.png
G:/Model_Compress/Dataset\training\0_206.png
G:/Model_Compress/Dataset\training\0_207.png
G:/Model_Compress/Dataset\training\0_208.png
G:/Model_Compress/Dataset\training\0_209.png
G:/Model_Compress/Dataset\training\0_210.png
G:/Model_Compress/Dataset\training\0_211.png
G:/Model_Compress/Dataset\training\0_212.png
G:/Model_Compress/Dataset\training\0_213.png
G:/Model_Compress/Dataset\training\0_214.png
G:/Model_Compress/Dataset\training\0_215.png
G:/Model_Compress/Dataset\training\0_216.png
G:/Model_Compress/Dataset\training\0_217.png
G:/Model_Compress/Dataset\training\0_218.png
G:/Model_Compress/Dataset\training\0_219.png
G:/Model_Compress/Dataset\training\0_220.png
G:/Model_Compress/Dataset\training\0_221.p

G:/Model_Compress/Dataset\training\1_72.png
G:/Model_Compress/Dataset\training\1_73.png
G:/Model_Compress/Dataset\training\1_74.png
G:/Model_Compress/Dataset\training\1_75.png
G:/Model_Compress/Dataset\training\1_76.png
G:/Model_Compress/Dataset\training\1_77.png
G:/Model_Compress/Dataset\training\1_78.png
G:/Model_Compress/Dataset\training\1_79.png
G:/Model_Compress/Dataset\training\1_8.png
G:/Model_Compress/Dataset\training\1_80.png
G:/Model_Compress/Dataset\training\1_81.png
G:/Model_Compress/Dataset\training\1_82.png
G:/Model_Compress/Dataset\training\1_83.png
G:/Model_Compress/Dataset\training\1_84.png
G:/Model_Compress/Dataset\training\1_85.png
G:/Model_Compress/Dataset\training\1_86.png
G:/Model_Compress/Dataset\training\1_87.png
G:/Model_Compress/Dataset\training\1_88.png
G:/Model_Compress/Dataset\training\1_89.png
G:/Model_Compress/Dataset\training\1_9.png
G:/Model_Compress/Dataset\training\1_90.png
G:/Model_Compress/Dataset\training\1_91.png
G:/Model_Compress/Dataset\training

In [4]:
print("Size of training data = {}".format(len(train_x)))

Size of training data = 400


In [6]:
# 数据增强

train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),  
    transforms.RandomRotation(15), 
    transforms.ToTensor(),   
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),                                    
    transforms.ToTensor(),
])
class ImgDataset(Dataset):
    def __init__(self, x, y=None, transform=None):
        self.x = x
        self.y = y
        if y is not None:
            self.y = torch.LongTensor(y)
        self.transform = transform
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        X = self.x[index]
        if self.transform is not None:
            X = self.transform(X)
        if self.y is not None:
            Y = self.y[index]
            return X, Y
        else:
            return X

In [7]:
batch_size = 32
train_set    = ImgDataset(train_x[:250],    train_y[:250],    train_transform)
val_set      = ImgDataset(train_x[250:350], train_y[250:350], test_transform)
test_set     = ImgDataset(train_x[350:],    train_y[350:],    test_transform)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_set,   batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_set,  batch_size=batch_size, shuffle=True)

In [8]:
class TeacherNet(nn.Module):
    def __init__(self):
        super(TeacherNet, self).__init__()
        bandwidth = [16, 32, 64, 128, 256, 256, 256, 256]     
        self.cnn = nn.Sequential(
                nn.Sequential(
                    # 3, 16, 3, 1, 1
                    nn.Conv2d(3, bandwidth[0], 3, 1, 1),
                    nn.BatchNorm2d(bandwidth[0]),
                    nn.ReLU6(),
                    nn.MaxPool2d(2, 2, 0),
                ),
       
                nn.Sequential(
                    nn.Conv2d(bandwidth[0], bandwidth[1], 3, 1, 1),
                    nn.BatchNorm2d(bandwidth[1]),
                    nn.ReLU6(),
                    nn.MaxPool2d(2, 2, 0),
                ),

                nn.Sequential(
                    nn.Conv2d(bandwidth[1], bandwidth[2], 3, 1, 1),
                    nn.BatchNorm2d(bandwidth[2]),
                    nn.ReLU6(),
                    nn.MaxPool2d(2, 2, 0),
                ),

                nn.Sequential(
                    nn.Conv2d(bandwidth[2], bandwidth[3], 3, 1, 1),
                    nn.BatchNorm2d(bandwidth[3]),
                    nn.ReLU6(),
                    nn.MaxPool2d(2, 2, 0),
                ),


                nn.Sequential(
                    nn.Conv2d(bandwidth[3], bandwidth[7], 3, 1, 1),
                    nn.BatchNorm2d(bandwidth[4]),
                    nn.ReLU6(),
                ),

                nn.Sequential(
                    nn.Conv2d(bandwidth[4], bandwidth[5], 3, 1, 1),
                    nn.BatchNorm2d(bandwidth[5]),
                    nn.ReLU6(),
                ),

                nn.Sequential(
                    nn.Conv2d(bandwidth[5], bandwidth[6], 3, 1, 1),
                    nn.BatchNorm2d(bandwidth[6]),
                    nn.ReLU6(),
                ),

                nn.Sequential(
                    nn.Conv2d(bandwidth[6], bandwidth[7], 3, 1, 1),
                    nn.BatchNorm2d(bandwidth[6]),
                    nn.ReLU6(),
                ),
                nn.AdaptiveAvgPool2d((1, 1)),
            )
        self.fc = nn.Sequential(
            nn.Linear(bandwidth[7], 2),
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

In [12]:
# model = TeacherNet()
model = model.cuda()
loss = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) 
num_epoch = 100

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    model.train() 
    for i, data in enumerate(train_loader):
        optimizer.zero_grad() 
        train_pred = model(data[0].cuda())
        label = torch.nn.functional.one_hot(data[1], 2).float().cuda()
        batch_loss = loss(train_pred, label) 
        batch_loss.backward() 
        optimizer.step() 

        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        train_loss += batch_loss.item()

    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            val_pred = model(data[0].cuda())
            label = torch.nn.functional.one_hot(data[1], 2).float().cuda()
            batch_loss = loss(val_pred, label)
            val_acc += np.sum(np.argmax(val_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
            val_loss += batch_loss.item()

        # 观察模型精度
        if (epoch+1) % 20 == 0 :
            print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
                (epoch + 1, num_epoch, time.time()-epoch_start_time, \
                 train_acc/train_set.__len__(), train_loss/train_set.__len__(), val_acc/val_set.__len__(), val_loss/val_set.__len__()))

[020/100] 0.84 sec(s) Train Acc: 0.992000 Loss: 0.000781 | Val Acc: 0.910000 loss: 0.014304
[040/100] 0.86 sec(s) Train Acc: 0.992000 Loss: 0.000357 | Val Acc: 0.880000 loss: 0.020800
[060/100] 0.83 sec(s) Train Acc: 0.996000 Loss: 0.000275 | Val Acc: 0.930000 loss: 0.021536
[080/100] 0.84 sec(s) Train Acc: 0.996000 Loss: 0.000235 | Val Acc: 0.890000 loss: 0.028299
[100/100] 0.86 sec(s) Train Acc: 0.984000 Loss: 0.000943 | Val Acc: 0.900000 loss: 0.058774


In [13]:
old_weights = model.state_dict()
# torch.save(w1, "DP_landslide_Student.pth") 

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
model.eval()
preds = []
labels = []
with torch.no_grad():
    for i, data in enumerate(test_loader):
        test_pred = model(data[0].cuda())
        label = data[1].numpy()
        labels.extend(label)
        test_pred  = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        preds.extend(test_pred)
    matrix = confusion_matrix(labels, preds)
print(matrix)

## DepthWise  PointWise

In [10]:
class StudentNet(nn.Module):
    def __init__(self, base=16, width_mult=1):
        super(StudentNet, self).__init__()
        multiplier = [1, 2, 4, 8, 16, 16, 16, 16]
        # bandwidth: 每一层的通道数
        bandwidth = [ base * m for m in multiplier]
        # 第3层后面开始  prune
        for i in range(3, 7):
            bandwidth[i] = int(bandwidth[i] * width_mult)
        # bandwidth = [16, 32, 64, 128, 256, 256, 256, 256]     
        self.cnn = nn.Sequential(
                nn.Sequential(
                    # 3, 16, 3, 1, 1
                    nn.Conv2d(3, bandwidth[0], 3, 1, 1),
                    nn.BatchNorm2d(bandwidth[0]),
                    nn.ReLU6(),
                    nn.MaxPool2d(2, 2, 0),
                ),
       
                nn.Sequential(
                    nn.Conv2d(bandwidth[0], bandwidth[0], 3, 1, 1, groups=bandwidth[0]),
                    nn.BatchNorm2d(bandwidth[0]),
                    nn.ReLU6(),
                    nn.Conv2d(bandwidth[0], bandwidth[1], 1),
                    nn.MaxPool2d(2, 2, 0),
                ),

                nn.Sequential(
                    nn.Conv2d(bandwidth[1], bandwidth[1], 3, 1, 1, groups=bandwidth[1]),
                    nn.BatchNorm2d(bandwidth[1]),
                    nn.ReLU6(),
                    nn.Conv2d(bandwidth[1], bandwidth[2], 1),
                    nn.MaxPool2d(2, 2, 0),
                ),

                nn.Sequential(
                    nn.Conv2d(bandwidth[2], bandwidth[2], 3, 1, 1, groups=bandwidth[2]),
                    nn.BatchNorm2d(bandwidth[2]),
                    nn.ReLU6(),
                    nn.Conv2d(bandwidth[2], bandwidth[3], 1),
                    nn.MaxPool2d(2, 2, 0),
                ),


                nn.Sequential(
                    nn.Conv2d(bandwidth[3], bandwidth[3], 3, 1, 1, groups=bandwidth[3]),
                    nn.BatchNorm2d(bandwidth[3]),
                    nn.ReLU6(),
                    nn.Conv2d(bandwidth[3], bandwidth[4], 1),
                ),

                nn.Sequential(
                    nn.Conv2d(bandwidth[4], bandwidth[4], 3, 1, 1, groups=bandwidth[4]),
                    nn.BatchNorm2d(bandwidth[4]),
                    nn.ReLU6(),
                    nn.Conv2d(bandwidth[4], bandwidth[5], 1),
                ),

                nn.Sequential(
                    nn.Conv2d(bandwidth[5], bandwidth[5], 3, 1, 1, groups=bandwidth[5]),
                    nn.BatchNorm2d(bandwidth[5]),
                    nn.ReLU6(),
                    nn.Conv2d(bandwidth[5], bandwidth[6], 1),
                ),

                nn.Sequential(
                    nn.Conv2d(bandwidth[6], bandwidth[6], 3, 1, 1, groups=bandwidth[6]),
                    nn.BatchNorm2d(bandwidth[6]),
                    nn.ReLU6(),
                    nn.Conv2d(bandwidth[6], bandwidth[7], 1),
                ),
                nn.AdaptiveAvgPool2d((1, 1)),
            )
        self.fc = nn.Sequential(
            nn.Linear(bandwidth[7], 2),
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

In [11]:
model = StudentNet()
Input = torch.ones((2, 3, 128, 128))
output = model(Input)
ouput = model(Input)
print(output.size())

torch.Size([2, 2])


 ### 通道裁剪函数，根据 gamma 系数降序排列旧模型每一层的参数，每次选取前 95% 的通道放入新的模型中

In [14]:
def network_slimming(old_model, new_model):
    params = old_model.state_dict()
    new_params = new_model.state_dict()
    
    # selected_idx: 每一层筛选后的 index
    selected_idx = []
    # 一共有7层CNN，
    for i in range(8):
        # gamma 参数在cnn.{i}.1.weight內。
        importance = params[f'cnn.{i}.1.weight']
      
        old_dim = len(importance)
        new_dim = len(new_params[f'cnn.{i}.1.weight'])
        # 排序
        ranking = torch.argsort(importance, descending=True)
        # 把每层前 95%的  放入selected_idx中。
        selected_idx.append(ranking[:new_dim])

    now_processed = 1
    for (name, p1), (name2, p2) in zip(params.items(), new_params.items()):
        # CNN层
        if name.startswith('cnn') and p1.size() != torch.Size([]) and now_processed != len(selected_idx):
            # 遇到 PW 层，计数器 +1，表示该 block 参数已经裁剪了
            if name.startswith(f'cnn.{now_processed}.3'):
                now_processed += 1

            # PW 层的参数维度会受到上层 和 下层的 裁剪结果影响，所以要 特判
            if name.endswith('3.weight'):
                # 如果是最后一层的 cnn，則不需要 prune。
                if len(selected_idx) == now_processed:
                    new_params[name] = p1[:,selected_idx[now_processed-1]]
                # 其他层就根据上下层的 Index 移植参数
                # Conv2d(x,y,1)的weight shape是(y,x,1,1)，順序是反的。
                else:
                    new_params[name] = p1[selected_idx[now_processed]][:,selected_idx[now_processed-1]]
            else:
                new_params[name] = p1[selected_idx[now_processed]]
        # CNN以外层直接复制
        else:
            new_params[name] = p1

    # 重新加载 pruned 参数       
    new_model.load_state_dict(new_params)
    return new_model

In [16]:
def run_epoch(dataloader, update=True, alpha=0.5):
    
    total_num, total_hit, total_loss = 0, 0, 0
    
    for now_step, batch_data in enumerate(dataloader):
        
        optimizer.zero_grad()
        
        inputs, labels = batch_data
        inputs = inputs.cuda()
        labels = labels.cuda()
  
        logits = net(inputs)
        loss = criterion(logits, labels)
        if update:
            loss.backward()
            optimizer.step()

        total_hit += torch.sum(torch.argmax(logits, dim=1) == labels).item()
        total_num += len(inputs)
        total_loss += loss.item() * len(inputs)

    return total_loss / total_num, total_hit / total_num

net = model
now_width_mult = 1
criterion = nn.CrossEntropyLoss()
for i in range(5):
    now_width_mult *= 0.95
    new_net = StudentNet(width_mult=now_width_mult).cuda()
    params = net.state_dict()
    net = network_slimming(net, new_net)
    now_best_acc = 0
    for epoch in range(5):
        net.train()
        train_loss, train_acc = run_epoch(train_loader, update=True)
        net.eval()
        valid_loss, valid_acc = run_epoch(val_loader, update=False)
        
        if valid_acc > now_best_acc:
            now_best_acc = valid_acc
            torch.save(net.state_dict(), 'landslide_small_rate_{:.3f}.pth'.format(now_width_mult))
        print('rate {:6.4f} epoch {:>3d}: train loss: {:6.4f}, acc {:6.4f} valid loss: {:6.4f}, acc {:6.4f}'.format(now_width_mult, 
            epoch+1, train_loss, train_acc, valid_loss, valid_acc))


rate 0.9500 epoch   1: train loss: 0.0341, acc 0.9840 valid loss: 0.8608, acc 0.9000
rate 0.9500 epoch   2: train loss: 0.0374, acc 0.9880 valid loss: 1.2101, acc 0.8800
rate 0.9500 epoch   3: train loss: 0.0684, acc 0.9720 valid loss: 1.3934, acc 0.8700
rate 0.9500 epoch   4: train loss: 0.0588, acc 0.9840 valid loss: 1.5313, acc 0.8600
rate 0.9500 epoch   5: train loss: 0.0883, acc 0.9800 valid loss: 1.5771, acc 0.8600
rate 0.9025 epoch   1: train loss: 0.0158, acc 0.9960 valid loss: 1.5618, acc 0.8600
rate 0.9025 epoch   2: train loss: 0.0813, acc 0.9760 valid loss: 1.6030, acc 0.8600
rate 0.9025 epoch   3: train loss: 0.0490, acc 0.9880 valid loss: 1.5752, acc 0.8600
rate 0.9025 epoch   4: train loss: 0.0812, acc 0.9720 valid loss: 1.5254, acc 0.8700
rate 0.9025 epoch   5: train loss: 0.0079, acc 0.9960 valid loss: 1.5477, acc 0.8700
rate 0.8574 epoch   1: train loss: 0.0283, acc 0.9840 valid loss: 1.5372, acc 0.8600
rate 0.8574 epoch   2: train loss: 0.0274, acc 0.9920 valid loss:

In [ ]:
pruned_weights = new_net.state_dict()
torch.save(w2, "pruned_DP_landslide.pth")

In [ ]:
print(new_net)